In [1]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


In [5]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)

# each review is encoded as a sequence of word indexes
# indexed by overall frequency in the dataset
# output is 0 (negative) or 1 (positive) 
imdb = tf.keras.datasets.imdb.load_data(num_words=max_features)
(raw_input_train, y_train), (raw_input_test, y_test) = imdb

In [6]:
# tf.keras.datasets.imdb.load_data?

In [7]:
y_train.min()

0

In [8]:
y_train.max()

1

In [9]:
# 25000 texts
len(raw_input_train)

25000

In [10]:
# first text has 218 words
len(raw_input_train[0])

218

In [11]:
raw_input_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [12]:
# tf.keras.preprocessing.sequence.pad_sequences?

In [13]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

input_train = tf.keras.preprocessing.sequence.pad_sequences(raw_input_train, maxlen=maxlen)
input_test = tf.keras.preprocessing.sequence.pad_sequences(raw_input_test, maxlen=maxlen)

In [14]:
input_train.shape, input_test.shape, y_train.shape, y_test.shape

((25000, 500), (25000, 500), (25000,), (25000,))

In [15]:
# left padded with zeros
# As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.
input_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## GRU

In [30]:
# Batch Normalization:
# https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c
# https://www.quora.com/Why-does-batch-normalization-help

drop_out = 0.15

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 32))
# for embedding: 32*2 (“standard deviation” parameter (gamma), “mean” parameter (beta)) trainable parameters
# and 32*2 (moving_mean and moving_variance) non-trainable parameters
model.add(tf.keras.layers.BatchNormalization())  
model.add(tf.keras.layers.Dropout(drop_out))

# 6240 / 32 = 195 parameters per gru unit
model.add(tf.keras.layers.GRU(32))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(drop_out))

# binary classifier
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 32)          320000    
_________________________________________________________________
batch_normalization_11 (Batc (None, None, 32)          128       
_________________________________________________________________
dropout_19 (Dropout)         (None, None, 32)          0         
_________________________________________________________________
gru_10 (GRU)                 (None, 32)                6240      
_________________________________________________________________
batch_normalization_12 (Batc (None, 32)                128       
_________________________________________________________________
dropout_20 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total para

In [31]:
batch_size = 1000

%time history = model.fit(input_train, y_train, epochs=25, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 15s 750us/step - loss: 0.8333 - acc: 0.5127 - val_loss: 0.6908 - val_acc: 0.5236
Epoch 2/25
20000/20000 [==============================] - 13s 664us/step - loss: 0.7323 - acc: 0.5456 - val_loss: 0.6891 - val_acc: 0.5410
Epoch 3/25
20000/20000 [==============================] - 14s 705us/step - loss: 0.7037 - acc: 0.5627 - val_loss: 0.6874 - val_acc: 0.5444
Epoch 4/25
20000/20000 [==============================] - 15s 769us/step - loss: 0.6777 - acc: 0.5876 - val_loss: 0.6846 - val_acc: 0.5540
Epoch 5/25
20000/20000 [==============================] - 14s 702us/step - loss: 0.6631 - acc: 0.6059 - val_loss: 0.6800 - val_acc: 0.5750
Epoch 6/25
20000/20000 [==============================] - 15s 735us/step - loss: 0.6406 - acc: 0.6285 - val_loss: 0.6722 - val_acc: 0.5994
Epoch 7/25
20000/20000 [==============================] - 16s 786us/step - loss: 0.6153 - acc: 0.6535 - val_loss: 0.6

In [32]:
train_loss, train_accuracy = model.evaluate(input_train, y_train, batch_size=batch_size)
train_accuracy

25000/25000 [==============================] - 5s 190us/step


0.9404799914360047

In [33]:
test_loss, test_accuracy = model.evaluate(input_test, y_test, batch_size=batch_size)
test_accuracy

25000/25000 [==============================] - 5s 186us/step


0.8703200078010559

In [34]:
# precition
model.predict(input_test[0:5])

array([[0.04822115],
       [0.9997733 ],
       [0.99491125],
       [0.8396635 ],
       [0.9998455 ]], dtype=float32)

In [35]:
# ground truth
y_test[0:5]

array([0, 1, 1, 0, 1], dtype=int64)